In [1]:
from cpsat_autotune import (
    tune_for_quality_within_timelimit,
    import_model,
    tune_time_to_optimal,
)

/home/krupke/anaconda3/envs/mo312/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model = import_model("models/model_proto_-3217353582387555034.pb")

In [3]:
# tune_for_quality_within_timelimit(models[0], 3, 10, "minimize", n_samples_per_param=2, max_samples_per_param=5, n_trials=100)
tune_time_to_optimal(
    model,
    timelimit_in_s=6,
    n_samples_per_param=10,
    max_samples_per_param=30,
    n_trials=50,
)

[I 2024-08-14 22:10:40,471] A new study created in memory with name: no-name-d1500e67-41b0-41a7-a651-25ca9f98594c


Computing baseline


[I 2024-08-14 22:11:12,990] Trial 0 finished with value: -1.0838156666666665 and parameters: {'repair_hint': False, 'use_lb_relax_lns': False, 'preferred_variable_order': 0, 'use_erwa_heuristic': False, 'linearization_level': 1, 'fp_rounding': 3, 'randomize_search': False, 'diversify_lns_params': False, 'add_objective_cut': False, 'use_objective_lb_search': False, 'use_objective_shaving_search': False, 'search_branching': 0, 'cut_level': 1, 'max_all_diff_cut_size': 1, 'symmetry_level': 2, 'max_presolve_iterations': 2, 'cp_model_probing_level': 2, 'presolve_probing_deterministic_time_limit': 4, 'presolve_bve_threshold': 1, 'ignore_subsolvers:default_lp': False, 'ignore_subsolvers:fixed': False, 'ignore_subsolvers:no_lp': False, 'ignore_subsolvers:max_lp': False, 'ignore_subsolvers:pseudo_costs': False, 'ignore_subsolvers:reduced_costs': False, 'ignore_subsolvers:quick_restart': False, 'ignore_subsolvers:quick_restart_no_lp': False, 'ignore_subsolvers:lb_tree_search': False, 'ignore_subs

Baseline: [-1.210967, -1.437454, -1.104262, -0.914416, -1.037159, -1.160257, -1.054899, -1.010246, -0.73242, -1.02839, -1.060057, -1.247562, -0.90201, -0.779221, -1.244729, -1.164658, -0.971829, -1.260762, -0.887293, -1.086368, -1.384286, -1.117178, -0.984919, -1.10395, -0.933098, -1.10571, -1.285413, -1.210084, -0.852139, -1.242734]


[I 2024-08-14 22:11:19,003] Trial 1 finished with value: -60.0 and parameters: {'repair_hint': False, 'use_lb_relax_lns': False, 'preferred_variable_order': 1, 'use_erwa_heuristic': True, 'linearization_level': 0, 'fp_rounding': 1, 'randomize_search': True, 'diversify_lns_params': False, 'add_objective_cut': True, 'use_objective_lb_search': True, 'use_objective_shaving_search': True, 'search_branching': 1, 'cut_level': 1, 'max_all_diff_cut_size': 1, 'symmetry_level': 0, 'max_presolve_iterations': 1, 'cp_model_probing_level': 2, 'presolve_probing_deterministic_time_limit': 1, 'presolve_bve_threshold': 2, 'ignore_subsolvers:default_lp': False, 'ignore_subsolvers:fixed': True, 'ignore_subsolvers:no_lp': True, 'ignore_subsolvers:max_lp': False, 'ignore_subsolvers:pseudo_costs': True, 'ignore_subsolvers:reduced_costs': False, 'ignore_subsolvers:quick_restart': True, 'ignore_subsolvers:quick_restart_no_lp': False, 'ignore_subsolvers:lb_tree_search': False, 'ignore_subsolvers:probing': False}